# Assignment 1c Notebook: Constraint Satisfaction EA
This notebook further iterates on the progress made in Assignment 1b and will guide you through the first part of this two-part assignment: building a constraint satisfaction EA. Similar to the last assignment, you should copy over the following files:
* 1a_notebook.ipynb
* 1b_notebook.ipynb
* baseEvolution.py
* binaryGenotype.py
* selection.py

*Be careful* to not copy over functions relating to the provided fitness functions, GPac, and static agents. We may have changed those and we want you to have the versions that were provided with this repo.

As usual, be sure to **read all of this notebook** and you can start by executing the next cell.

In [ ]:
# Configure this notebook to automatically reload modules as they're modified
# https://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 12.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'

# Note: we're skipping the plotMaze function from the other notebooks, but 
# you're welcome to copy and paste it here if you want to look at mazes in-notebook.

print('The first cell has been executed!')

## Constraint satisfaction
You may recall from the lectures that there are multiple ways to handle constraints in an EA. You can employ the following techniques:
* ignore constraints
* kill invalid offspring
* feasible phenotype mapping decoder
* repair function
* feasible solution space closed under variation operators
* penalty function

For this assignment, we're going to guide you through the process of making a  constraint satisfaction EA using a penalty function. This means that fitness will reflect not only the performance of the map but also how well the solution satisfies the constraint of a valid GPac map. Consequently, evolution will (hopefully) evolve maps that are increasingly difficult for pac-man *and* increasingly valid.

At this point, you might be asking the very sane question: *isn't there already a repair function?* 

Well... yes, GPac has constraints such that the mechanics of the game simply don't work if there's not a path between the spawn of pac-man and the ghosts or if pills/fruit can spawn in unreachable locations. You would typically use *either* a repair function or a penalty function, but in this assignment we're going to have you calculate a penalty based on the number of walls the repair function had to modify to create a valid GPac map. This is essentially evolving to minimize the impact (and any bias) of the repair function.

The following cell demonstrates how to determine the number of repairs made. Note that the behavior of `repair_and_test_map` is different from 1b because the config file contains an extra parameter assignment `return_repair_count = True`.

In [ ]:
from selection import *
from snakeeyes import readConfig
from binaryGenotype import binaryGenotype
from fitness import repair_and_test_map

config = readConfig('./configs/green1c0_config.txt', globalVars=globals(), localVars=locals())

testSolution = binaryGenotype()
testSolution.randomInitialization(**config['initialization_kwargs'])

testSolution.rawFitness, testSolution.log, numRepairs = repair_and_test_map(testSolution.gene, **config['fitness_kwargs'])

game_log_path = 'worldFiles/1cnotebooktest.txt'
with open(game_log_path, 'w') as f:
	[f.write(f'{line}\n') for line in testSolution.log]
    
print(f"The solution's unpenalized (raw) fitness was {testSolution.rawFitness} and the game log was written to {game_log_path}")
print(f'The number of repairs made to the genotype: {numRepairs}')

Note that we assigned a new member variable `rawFitness` with the unpenalized fitness returned by `repair_and_test_map`. This is so that you can perform stats using *this* value to compare against your implementation from 1b and the results will be comparable since both algorithms report the same (unpenalized) notion of fitness. To be abundantly clear: **it is invalid to compare penalized with unpenalized fitness in your statistical analysis!**

Now that we know the number of repairs made, we can calculate penalized fitness and assign it to the individual as follows:

In [ ]:
penalty_coefficient = config['fitness_kwargs']['penalty_coefficient'] # we'll present a more graceful way of accessing this

testSolution.fitness = testSolution.rawFitness - penalty_coefficient*numRepairs

print(f"The solution's penalized fitness was {testSolution.fitness}")

del testSolution # to discourage haphazard copypasta

Now that we've demonstrated how to calculate penalized fitness and store unpenalized fitness for logging purposes, it's your turn to write a function that evaluates an input population. Notice that the inputs are a little different than assignment 1b and this function will strip off the `penalty_coefficient` parameter and pass all other keyword arguments into a dictionary you can use with `**fitness_kwargs`. We'll call this function a little differently as a result (see the cell after next for more details).

#### Note on YELLOW 1 Deliverable
If you're attempting the YELLOW 1 deliverable, complete this notebook and notebook `1c_notebook1.ipynb`, return to this cell, and then modify the following cell to use a self-adaptive penalty coefficient. Note however that you should not delete or completely overwrite the original funcitonality required for the green deliverables. In other words, the later cells in this notebook should work correctly with the config files `green1c0_config.txt1` *and* `yellow1c_config.txt`.

In [ ]:
# evaluate the population and assign fitness, rawFitness, and logs as described above
def evaluate_population(population, penalty_coefficient, **fitness_kwargs):
    pass

In [ ]:
import statistics

examplePopulation = binaryGenotype.initialization(10, **config['initialization_kwargs'])

# calling your function to test things out (this line is different than notebook 1b)
evaluate_population(examplePopulation, **config['fitness_kwargs'])

print(f'Individuals with unassigned fitness: {len([individual.fitness for individual in examplePopulation if individual.fitness is None])}')
print(f'Number of fitness evaluations performed: {len([individual.fitness for individual in examplePopulation if individual.fitness is not None])}')
print(f'Average fitness of population: {statistics.mean([individual.fitness for individual in examplePopulation])}')
maxFitness = max([individual.fitness for individual in examplePopulation])
print(f'Best fitness in population: {maxFitness}')
print(f'Average unpenalized (raw) fitness of population: {statistics.mean([individual.rawFitness for individual in examplePopulation])}')
maxRawFitness = max([individual.rawFitness for individual in examplePopulation])
print(f'Best unpenalized (raw) fitness in population: {maxRawFitness}')
bestLog = None
for individual in examplePopulation:
    if individual.fitness == maxFitness:
        bestLog = individual.log
        break

print(f'Found log of highest scoring individual? {bestLog is not None}')
with open(game_log_path, 'w') as f:
	[f.write(f'{line}\n') for line in bestLog]
    
print(f"The log of the most fit individual was written to {game_log_path}")

del examplePopulation # to discourage haphazard copypasta

## Assembling your EA
Now you get to use the framework you implemented in Assignment 1b to *just build an EA*. Since your selection operators already function with negative fitness and you assigned penalized fitness to the `fitness` member variable of the individuals in your populations, your existing EA framework should work without the need for modification. Neat!

As usual, we'll have you start out by performing a single run of your EA that performs 2,000 evaluations. Note that this cell can be nearly identical to a cell from 1b except the call to `evaluate_population` will differ slightly.

In [ ]:
number_evaluations = 2000

# You can parse different configuration files here as necessary
config = readConfig('./configs/green1c0_config.txt', globalVars=globals(), localVars=locals())

# implement your EA here


Now that you've tested an implmentation of a single run, implement code to perform 30 runs of your EA search that each contain 2,000 evaluations. For each generation of each run, log the mean and best fitness of the current population as well as the number of fitness evaluations performed so far (including the initial population). Also for each run, record the best **unpenalized (raw) fitness** found during the run.

In [ ]:
number_runs = 30
number_evaluations = 2000

# You can parse different configuration files here as necessary
config = readConfig('./configs/green1c0_config.txt', globalVars=globals(), localVars=locals())

# implement your multi-run experiment here


## Report
Using the data you've collected from your 30 run experiment, average per-generation across all runs to find the average mean and maximum population fitnesses across 30 runs. Using this data, produce a plot that shows the mean and best fitness per number of fitness evaluations averaged over 30 runs. This is the same plot as Assignment 1b. Include this in your report along with any statistical analysis or additional requested components from the assignment description. Statistical analysis should consist of a comparison between best per-run **unpenalized fitness** values of your 1b and 1c EA searches. Use data generated with the 1b notebook to compare against your 1c EA results.